In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import vgg16
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from sklearn.model_selection import train_test_split

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Specify the path to your dataset
data_path =  r"C:\Users\Capstone\Desktop\hieroglyph-image-recognition-vgg16\Augmented_images"

# Define the transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images to a specific size
    transforms.ToTensor(),  # Convert the images to tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the image tensors
])

# Load the dataset and split into train, validation, and test sets
dataset = ImageFolder(data_path, transform=transform)
train_data, test_data, train_labels, test_labels = train_test_split(dataset.samples, dataset.targets, test_size=0.2)
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.2)

# Create data loaders for training, validation, and test sets
batch_size = 32

train_dataset = ImageFolder(data_path, transform=transform)
train_dataset.samples = train_data
train_dataset.targets = train_labels
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = ImageFolder(data_path, transform=transform)
val_dataset.samples = val_data
val_dataset.targets = val_labels
val_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

test_dataset = ImageFolder(data_path, transform=transform)
test_dataset.samples = test_data
test_dataset.targets = test_labels
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load the pre-trained VGG16 model
model = vgg16(pretrained=True)

# Freeze the parameters of the pre-trained layers
for param in model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer for the number of classes in your dataset
num_classes = len(train_dataset.classes)
model.classifier[6] = nn.Linear(4096, num_classes)

# Move the model to the device
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.classifier.parameters(), lr=0.001, momentum=0.9)

# Set the number of epochs
num_epochs = 24

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for images, labels in train_data_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_data_loader)

    # Evaluation loop - Validation set
    model.eval()  # Set the model to evaluation mode
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for images, labels in val_data_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    val_accuracy = correct_predictions / total_predictions

    # Print the epoch statistics
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

# Evaluation on the test set
model.eval()  # Set the model to evaluation mode
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for images, labels in test_data_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

test_accuracy = correct_predictions / total_predictions

print(f"Test Accuracy: {test_accuracy:.4f}")

Epoch [1/24], Train Loss: 5.9545, Val Accuracy: 0.2236
Epoch [2/24], Train Loss: 4.8834, Val Accuracy: 0.3584
Epoch [3/24], Train Loss: 4.1751, Val Accuracy: 0.4586
Epoch [4/24], Train Loss: 3.6792, Val Accuracy: 0.5229
Epoch [5/24], Train Loss: 3.3104, Val Accuracy: 0.5698
Epoch [6/24], Train Loss: 3.0305, Val Accuracy: 0.6041
Epoch [7/24], Train Loss: 2.8066, Val Accuracy: 0.6361
Epoch [8/24], Train Loss: 2.6286, Val Accuracy: 0.6451
Epoch [9/24], Train Loss: 2.4802, Val Accuracy: 0.6657
Epoch [10/24], Train Loss: 2.3603, Val Accuracy: 0.6753
Epoch [11/24], Train Loss: 2.2511, Val Accuracy: 0.6917
Epoch [12/24], Train Loss: 2.1535, Val Accuracy: 0.7046
Epoch [13/24], Train Loss: 2.0724, Val Accuracy: 0.7090
Epoch [14/24], Train Loss: 2.0069, Val Accuracy: 0.7146
Epoch [15/24], Train Loss: 1.9401, Val Accuracy: 0.7283
Epoch [16/24], Train Loss: 1.8876, Val Accuracy: 0.7323
Epoch [17/24], Train Loss: 1.8305, Val Accuracy: 0.7383
Epoch [18/24], Train Loss: 1.7781, Val Accuracy: 0.7425
E

In [ ]:
torch.save(model.state_dict(),'hieroglyph_vgg_16-2.pth')